## Análise dos dados do dataset

In [4]:
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 5)
pd.set_option('display.max_columns',100)

In [5]:
df = pd.read_csv('../dataset/B2W-Reviews01.csv')

c:\Users\STEFA\Anaconda3\envs\apolo\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132371,2018-05-31 23:46:48,e6fb0b19277d01c2a300c7837a105f3c369377e92f9c19...,6774907,Etiquetas Jurídicas Vade Mecum - Marca Fácil,marca facil,Papelaria,Material de Escritório,O produto não é bom.,1,No,"Material fraco, poderia ser melhor. Ficou deve...",1991.0,M,RJ
132372,2018-05-31 23:50:33,ea9467aa73271fb4f68c04f4dd4f4eef304d6ee85441fb...,114081902,Painel de Fotos Bee Colection Rue Bac (74x94x3...,kapos,Decoração,Painel de Fotos,Produto não entregue,1,No,"Comprei esse produto, quando chegou estava com...",1982.0,F,ES
